AiCore Multinational Retail Data Centralisation Project
Milestone 4: Query the data using QL

Author: Maxwell Acha

Date: 2023-11-20

In [5]:
# Import libraries and connect to database

import pandas as pd
from data_ingestion.database_utils import DatabaseConnector
from sqlalchemy import text

db_connector = DatabaseConnector("data_ingestion\db_local_creds.yaml")
db_connector.init_db_engine(autocommit=True)
engine = db_connector.engine


def execute(sql):
    '''Helper function for executing SQL and printing results.

    Arguments:
        sql (string): SQL query
    
    Returns:
        None'''
        
    with engine.connect() as con:
        df = pd.read_sql_query(sql=text(sql), con=con)
        print(df)

Loading database credentials from file: data_ingestion\db_local_creds.yaml
Connecting to database: postgres


In [6]:
print(df.head())

NameError: name 'df' is not defined

: 

1. How many stores does the business have and in which countries?

In [2]:
sql = '''
SELECT country_code, count(*) AS total_no_stores
FROM dim_store_details
GROUP BY country_code
ORDER BY country_code;
'''

execute(sql)

  country_code  total_no_stores
0           DE              141
1           GB              266
2           US               34


2. Which locations currently have the most stores?

In [3]:
sql = '''
SELECT locality, count(*) AS total_no_stores
FROM dim_store_details
GROUP BY locality
ORDER BY total_no_stores DESC
LIMIT 10;
'''

execute(sql)

       locality  total_no_stores
0    Chapletown               14
1        Belper               13
2        Bushey               12
3        Exeter               11
4  High Wycombe               10
5    Rutherglen               10
6      Arbroath               10
7      Surbiton                9
8       Lancing                9
9      Aberdeen                9


In [4]:
print(df.head())

NameError: name 'df' is not defined

3. Which months produce the highest average cost of sales?

In [5]:
sql = '''
SELECT
	ROUND(CAST(SUM(spend) AS numeric), 2) AS total_sales,
	month
FROM (
	SELECT 
		ord.product_quantity * prod.product_price AS spend,
		dt.month
	FROM orders_table ord
	INNER JOIN dim_date_times dt
		ON ord.date_uuid = dt.date_uuid
	INNER JOIN dim_products prod
		ON ord.product_code = prod.product_code
) x
GROUP BY month
ORDER BY total_sales DESC;
'''

execute(sql)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column prod.product_code does not exist
LINE 13:   ON ord.product_code = prod.product_code
                                 ^
HINT:  Perhaps you meant to reference the column "ord.product_code".

[SQL: 
SELECT
	ROUND(CAST(SUM(spend) AS numeric), 2) AS total_sales,
	month
FROM (
	SELECT 
		ord.product_quantity * prod.product_price AS spend,
		dt.month
	FROM orders_table ord
	INNER JOIN dim_date_times dt
		ON ord.date_uuid = dt.date_uuid
	INNER JOIN dim_products prod
		ON ord.product_code = prod.product_code
) x
GROUP BY month
ORDER BY total_sales DESC;
]
(Background on this error at: https://sqlalche.me/e/14/f405)

4. How many sales are coming from online?

In [6]:
sql = '''
SELECT
	COUNT(*) AS numbers_of_sales,
	SUM(product_quantity) AS product_quantity_count,
	location
FROM (
	SELECT 
		ord.product_quantity,
		CASE
			WHEN st.store_type = 'Web Portal' THEN 'Web'
			ELSE 'Offline'
		END AS location
	FROM orders_table ord
	INNER JOIN dim_store_details st
		ON ord.store_code = st.store_code
) x
GROUP BY location
ORDER BY location DESC;
''' 

execute(sql)

   numbers_of_sales  product_quantity_count location
0             26957                107739.0      Web
1             93166                374047.0  Offline


5. What percentage of sales come through each type of store?

In [13]:
sql = ''' 
SELECT 
    dim_store_details.store_type,
    COUNT(*) AS total_sales,
    ROUND(COUNT(*) * 100.0/SUM(COUNT(*)) OVER (),2) AS percentage_total -- calculates the ratio of the count of rows for each sales type to the total count of all rows in the result set.
    
  FROM 
    orders_table 
    INNER JOIN dim_store_details ON orders_table.store_code = dim_store_details.store_code
  GROUP BY
    dim_store_details.store_type
  ORDER BY percentage_total DESC;

''' 

execute(sql)

    store_type  total_sales  percentage_total
0        Local        53894             44.87
1   Web Portal        26957             22.44
2  Super Store        18779             15.63
3   Mall Kiosk        10762              8.96
4       Outlet         9731              8.10


6. Which months produced the highest sales?

In [8]:
sql = ''' 
SELECT
	ROUND(CAST(SUM(sale) AS NUMERIC), 2) AS total_sales,
	year,
	month
FROM (
	SELECT 
		dt.year,
		dt.month,
		ord.product_quantity * prod.product_price AS sale
	FROM orders_table ord
	INNER JOIN dim_products prod
		ON ord.product_code = prod.product_code
	INNER JOIN dim_date_times dt
		ON ord.date_uuid = dt.date_uuid
) x
GROUP BY year, month
ORDER BY total_sales DESC
LIMIT 10;
''' 

execute(sql)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column prod.product_code does not exist
LINE 13:   ON ord.product_code = prod.product_code
                                 ^
HINT:  Perhaps you meant to reference the column "ord.product_code".

[SQL:  
SELECT
	ROUND(CAST(SUM(sale) AS NUMERIC), 2) AS total_sales,
	year,
	month
FROM (
	SELECT 
		dt.year,
		dt.month,
		ord.product_quantity * prod.product_price AS sale
	FROM orders_table ord
	INNER JOIN dim_products prod
		ON ord.product_code = prod.product_code
	INNER JOIN dim_date_times dt
		ON ord.date_uuid = dt.date_uuid
) x
GROUP BY year, month
ORDER BY total_sales DESC
LIMIT 10;
]
(Background on this error at: https://sqlalche.me/e/14/f405)

7. What is our staff headcount?

In [9]:
sql = ''' 
SELECT 
	SUM(staff_numbers) AS total_staff_numbers,
	country_code
FROM dim_store_details
GROUP BY country_code
ORDER BY total_staff_numbers DESC;
'''

execute(sql)

   total_staff_numbers country_code
0              13132.0           GB
1               6054.0           DE
2               1304.0           US


8. Which German store type is selling the most?

In [11]:
sql = ''' 
SELECT 
	ROUND(CAST(SUM(sale) AS NUMERIC), 2) AS total_sales,
	store_type,
	country_code
FROM (
SELECT
	st.store_type,
	st.country_code,
	ord.product_quantity * prod.product_price AS sale
FROM orders_table ord
INNER JOIN (
	SELECT store_code, store_type, country_code
	FROM dim_store_details 
	WHERE country_code = 'DE'
	) st
	ON ord.store_code = st.store_code
INNER JOIN dim_products prod
	ON ord.product_code = prod.product_code
) x
GROUP BY 
	store_type,
	country_code
ORDER BY 
	total_sales;
''' 

execute(sql)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column prod.product_code does not exist
LINE 19:  ON ord.product_code = prod.product_code
                                ^
HINT:  Perhaps you meant to reference the column "ord.product_code".

[SQL:  
SELECT 
	ROUND(CAST(SUM(sale) AS NUMERIC), 2) AS total_sales,
	store_type,
	country_code
FROM (
SELECT
	st.store_type,
	st.country_code,
	ord.product_quantity * prod.product_price AS sale
FROM orders_table ord
INNER JOIN (
	SELECT store_code, store_type, country_code
	FROM dim_store_details 
	WHERE country_code = 'DE'
	) st
	ON ord.store_code = st.store_code
INNER JOIN dim_products prod
	ON ord.product_code = prod.product_code
) x
GROUP BY 
	store_type,
	country_code
ORDER BY 
	total_sales;
]
(Background on this error at: https://sqlalche.me/e/14/f405)

9. How quickly is the company making sales?
Average time interval between sales; grouped by year.

In [12]:
sql = ''' 
SELECT
	year,
	--avg_datediff,
	'"hours": ' || TO_CHAR(avg_datediff, 'HH24') || 
	', "minutes": ' || TO_CHAR(avg_datediff, 'MI') ||
	', "seconds": ' || TO_CHAR(avg_datediff, 'SS') ||
	', "milliseconds": ' || TO_CHAR(avg_datediff, 'MS')
	AS actual_time_taken
FROM (
	SELECT
		year,
		AVG(datediff) AS avg_datediff
	FROM (
		SELECT
			year,
			dt,
			next_dt,
			next_dt - dt AS datediff
		FROM (
			SELECT 
				year,
				dt, 
				LEAD(dt, 1) OVER (ORDER BY dt) next_dt
			FROM (
				SELECT 
					year,
					CAST(year || '-' || month || '-' || day || ' ' || timestamp AS TIMESTAMP(1) WITHOUT TIME ZONE) dt
				FROM dim_date_times
			) concat_datetime
		) get_next_datetime
	) subtract_datetimes
	GROUP BY year
) get_average_interval
ORDER BY avg_datediff DESC
LIMIT 5;
'''

execute(sql)


   year                                  actual_time_taken
0  2013  "hours": 02, "minutes": 17, "seconds": 15, "mi...
1  1993  "hours": 02, "minutes": 15, "seconds": 40, "mi...
2  2002  "hours": 02, "minutes": 13, "seconds": 49, "mi...
3  2008  "hours": 02, "minutes": 13, "seconds": 03, "mi...
4  2022  "hours": 02, "minutes": 13, "seconds": 02, "mi...
